In [1]:
import os

In [4]:
pwd

'f:\\projects\\project-9 kidney disease\\kidney-disease-classification'

In [3]:
os.chdir('../')

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareBaseModelConfig:
    root_dir: Path 
    base_model_path: Path 
    updated_base_model_path: Path 
    params_image_size: int 
    params_learning_rate: float 
    params_include_top: bool
    params_weights: str 
    params_classes: int
    params_pooling: str

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import create_directories, read_yaml

In [24]:
class ConfigurationManager:
    def __init__(self, config_filepath="config/config.yaml", params_filepath="params.yaml"):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        create_directories([self.config.artifacts_root])

    def _create_config_for(self, model_name: str) -> PrepareBaseModelConfig:
        """Internal helper to map YAML to the Entity"""
        model_config = self.config.prepare_base_model[model_name]
        create_directories([model_config.dir])
        
        return PrepareBaseModelConfig(
            root_dir=Path(model_config.dir),
            base_model_path=Path(model_config.raw_path),
            updated_base_model_path=Path(model_config.updated_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_pooling=self.params.POOLING
        )

    # 2. THE EXPLICIT METHODS (The "Interface")
    def get_densenet121_config(self) -> PrepareBaseModelConfig:
        densenet_prepare_config = self._create_config_for('densenet121')
        return densenet_prepare_config

    def get_resnet50_config(self) -> PrepareBaseModelConfig:
        resnet_prepare_config = self._create_config_for('resnet50')
        return resnet_prepare_config

    def get_efficientnetb0_config(self) -> PrepareBaseModelConfig:
        efficientnet_prepare_config = self._create_config_for('efficientnetb0')
        return efficientnet_prepare_config

In [42]:
import os 
import urllib.request as request
import zipfile
from pathlib import Path
import tensorflow as tf
from cnnClassifier import logger

In [45]:
class prepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    def get_base_model(self, model_type: str):
        """Downloads the raw base model from keras applications """
        
        if model_type == 'densenet121':
            model_fn = tf.keras.applications.DenseNet121
        elif model_type == 'resnet50':
            model_fn = tf.keras.applications.ResNet50
        elif model_type == 'efficientnetb0':
            model_fn = tf.keras.applications.EfficientNetB0
            
        self.model = model_fn(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            pooling=self.config.params_pooling,
        )
        
        self.save_model(model = self.model, path = self.config.base_model_path)
        
    @staticmethod 
    def save_model(model: tf.keras.Model, path: Path):
        """ Saves the keras model to the path """
        str_path = str(path)
        model.save(str_path)
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till,learning_rate):
        """ Creates the full model with custom head """
        
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
                
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False
                
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(model.output)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        
        
        full_model.summary()
        print(f"Model Output Shape: {full_model.output_shape}")
        return full_model
    
    def update_base_model(self):
        """ Updates the base model with custom head """
        
        full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        
        self.save_model(model=full_model, path=self.config.updated_base_model_path)

In [46]:
def run_base_model_pipeline():
    try:
        config_manager = ConfigurationManager()
        
        # List of (method_to_call, model_string_for_if_statement)
        # Note: These strings MUST match your if/elif logic
        model_tasks = [
            (config_manager.get_densenet121_config, "densenet121"),
            (config_manager.get_resnet50_config, "resnet50"),
            (config_manager.get_efficientnetb0_config, "efficientnetb0")
        ]

        for get_config, model_name in model_tasks:
            print(f"--- Processing {model_name} ---")
            
            # 1. Clear RAM before starting a new model
            tf.keras.backend.clear_session()
            
            # 2. Get the config
            current_config = get_config()
            
            # 3. Run the Component
            prepare_model = prepareBaseModel(config=current_config)
            prepare_model.get_base_model(model_type=model_name)
            prepare_model.update_base_model()
            
            
            print(f"--- {model_name} setup complete! ---\n")
    except Exception as e:
    # Only this block runs if the code ACTUALLY crashes
        logger.info(f"Failed to process {model_name}: {e}")

# Run it
run_base_model_pipeline()


[2025-12-19 18:30:04,546: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-19 18:30:04,652: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-19 18:30:04,682: INFO: common: created directory at : artifacts]
--- Processing densenet121 ---
[2025-12-19 18:30:06,551: INFO: common: created directory at : artifacts/prepare_base_model/densenet121]
[2025-12-19 18:30:28,339: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
   